In [ ]:
# Tensorflowが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env TF_NUM_INTEROP_THREADS=1
%env TF_NUM_INTRAOP_THREADS=1

from tensorflow.config import threading
num_threads = 1
threading.set_inter_op_parallelism_threads(num_threads)
threading.set_intra_op_parallelism_threads(num_threads)

#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## ミニバッチ法のKerasによる実装
Tensorflow/KerasではMinibatch法が簡単に使えるようようになっています。
基礎編で使った2次元データとMLPモデルを例に実験します。

In [ ]:
# 二次元ガウス分布と一様分布
def get_dataset_2():
    from numpy.random import default_rng
    rng = default_rng(seed=0)  # 今回はデータセットの乱数を固定させます。

    num_signal = 100  # 生成するシグナルイベントの数
    num_background = 1000  # 生成するバックグラウンドイベントの数

    # データ点の生成
    ## 平均(x1,x2) = (1.0, 0.0)、分散=1の２次元ガウス分布
    x_sig = rng.multivariate_normal(mean=[1.0, 0],
                                    cov=[[1, 0], [0, 1]],
                                    size=num_signal)
    t_sig = np.ones((num_signal, 1))  # Signalは1にラベリング

    ## (-5, +5)の一様分布
    x_bg = rng.uniform(low=-5, high=5, size=(num_background, 2))
    t_bg = np.zeros((num_background, 1))  # Backgroundは0にラベリング

    # 2つのラベルを持つ学習データを1つにまとめる
    x = np.concatenate([x_sig, x_bg])
    t = np.concatenate([t_sig, t_bg])

    # データをランダムに並び替える
    p = rng.permutation(len(x))
    x, t = x[p], t[p]

    return x, t


# ラベル t={0,1}を持つデータ点のプロット
def plot_datapoint(x, t):
    # シグナル/バックグラウンドの抽出
    xS = x[t[:, 0] == 1]  # シグナルのラベルだけを抽出
    xB = x[t[:, 0] == 0]  # バックグラウンドのラベルだけを抽出

    # プロット
    plt.scatter(xS[:, 0], xS[:, 1], label='Signal', c='red', s=10)  # シグナルをプロット
    plt.scatter(xB[:, 0], xB[:, 1], label='Background', c='blue', s=10)  # バックグラウンドをプロット
    plt.xlabel('x1')  # x軸ラベルの設定
    plt.ylabel('x2')  # y軸ラベルの設定
    plt.legend()  # legendの表示
    plt.show()


# prediction関数 の等高線プロット (fill)
def plot_prediction(prediction, *args):
    # 等高線を描くためのメッシュの生成
    x1, x2 = np.mgrid[-5:5:100j, -5:5:100j]  # x1 = (-5, 5), x2 = (-5, 5) の範囲で100点x100点のメッシュを作成
    x1 = x1.flatten()  # 二次元配列を一次元配列に変換 ( shape=(100, 100) => shape(10000, ))
    x2 = x2.flatten()  # 二次元配列を一次元配列に変換 ( shape=(100, 100) => shape(10000, ))
    x = np.array([x1, x2]).T

    #  関数predictionを使って入力xから出力yを計算し、等高線プロットを作成
    y = prediction(x, verbose=0, *args)
    cs = plt.tricontourf(x[:, 0], x[:, 1], y.flatten(), levels=10)
    plt.colorbar(cs)


In [ ]:
# データ点の取得 (データセットのサイズは1100です。)
x, t = get_dataset_2()
print("データのサイズ", x.shape, t.shape)

まずは一度に全てのデータを使ってロスの計算を行うバッチ学習を試してみます。

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

# モデルの定義
model = Sequential([
    Input(shape=(2,)),
    Dense(units=64, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU。
    Dense(units=64, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU。
    Dense(units=64, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU。
    Dense(units=64, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU。
    Dense(units=64, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU。
    Dense(units=1, activation='sigmoid')  # ノード数が1の層を追加。活性化関数はシグモイド関数。
])

#  誤差関数としてクロスエントロピーを指定。最適化手法はadam
model.compile(loss='binary_crossentropy', optimizer='SGD')

# バッチ法によるトレーニング
_ = model.fit(
    x=x,
    y=t,
    batch_size=1100,  # バッチサイズ。一回のステップで1100行のデータ(全データ)を使うようにする。
    epochs=1,  # 学習のステップ数
    verbose=1,  # 1とするとステップ毎に誤差関数の値などが表示される
)

# プロット
## パーセプトロンの出力を等高線プロット
plot_prediction(model.predict)

## データ点をプロット
plot_datapoint(x, t)

次に1行ずつロスを計算するオンライン学習で学習を実行します。

In [ ]:
# モデルの定義
model = Sequential([
    Input(shape=(2,)),
    Dense(units=64, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU。
    Dense(units=64, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU。
    Dense(units=64, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU。
    Dense(units=64, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU。
    Dense(units=64, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU。
    Dense(units=1, activation='sigmoid')  # ノード数が1の層を追加。活性化関数はシグモイド関数。
])

#  誤差関数としてクロスエントロピーを指定。最適化手法はadam
model.compile(loss='binary_crossentropy', optimizer='SGD')

# オンライン法によるトレーニング
_ = model.fit(
    x=x,
    y=t,
    batch_size=1,  # バッチサイズ。一回のステップで1行のデータのみを使うようにする。
    epochs=1,  # 学習のステップ数
    verbose=1,  # 1とするとステップ毎に誤差関数の値などが表示される
)

# プロット
## パーセプトロンの出力を等高線プロット
plot_prediction(model.predict)

## データ点をプロット
plot_datapoint(x, t)

Kerasの出力するログを見比べるといくつか値が変わっています。

- 左端の"1/1", "1100/1100"は重みの更新のステップ数を表しています。バッチ法では全データを一度に使って重みの更新をしているため、1エポックあたりの重みの更新回数は1回です。一方で、オンライン学習は1行ごとに更新を行うため、合計10000回重みの更新がおこなわれます。
- 中程にある "XXms/step"は1回の重み更新にかかる時間を表しています。バッチ法では全データを使うため、1回の重みの更新に時間がかかります。オンライン学習では1行のみなので、1回の重み更新計算は高速です。
- その左の "XXs"はトータルでかかった時間を表します。オンライン学習では1ステップあたりの時間は短いですが、1100回重みの更新をする必要があるためトータルの時間はバッチ法よりも長くかかっています。
- 今回は全データを１回ずつ使用して(エポック数1で)学習したためバッチ法では学習が十分に進んでいません。エポック数(`fit`メソッドの中の`epochs`)を増やすことで学習をさらに進めることができます。

次にミニバッチ法を試してみましょう。バッチサイズは10としてみます。

In [ ]:
# モデルの定義
model = Sequential([
    Input(shape=(2,)),
    Dense(units=64, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU。
    Dense(units=64, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU。
    Dense(units=64, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU。
    Dense(units=64, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU。
    Dense(units=64, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU。
    Dense(units=1, activation='sigmoid')  # ノード数が1の層を追加。活性化関数はシグモイド関数。
])

#  誤差関数としてクロスエントロピーを指定。最適化手法はadam
model.compile(loss='binary_crossentropy', optimizer='SGD')

# オンライン法によるトレーニング
_ = model.fit(
    x=x,
    y=t,
    batch_size=10,  # バッチサイズ。一回のステップで10行のデータのみを使うようにする。
    epochs=1,  # 学習のステップ数
    verbose=1,  # 1とするとステップ毎に誤差関数の値などが表示される
)

# プロット
## パーセプトロンの出力を等高線プロット
plot_prediction(model.predict)

## データ点をプロット
plot_datapoint(x, t)

1回の重み更新で10イベントを処理するので、全体として110ステップ分処理することになりました。
振る舞いはバッチ学習とオンライン学習の中間くらいになっています。

今回はデータセットがシンプルだったため、オンライン学習でも問題なく学習が進みましたが、オンライン学習は学習が不安定になることが多いです。
そのため実際の深層学習モデル学習の際は、バッチサイズが32 ~ 2048程度のミニバッチ学習を使うことが多いです。